In [0]:
import math
import numpy as np

In [0]:
#Inputs
S_0 = 100
r = 0.05
sigma = 0.25
T = 1
step = 4
delta_T = T/step
u = math.exp(sigma*(delta_T)**0.5)
d = math.exp(-sigma*(delta_T)**0.5)
q = (math.exp(r*delta_T) - d)/(u - d)


In [125]:
print(u)
print(d)
print(q)
print(delta_T)

1.1331484530668263
0.8824969025845955
0.5189736457076521
0.25


In [127]:
tree = np.zeros((step + 1,step + 1))
tree[0,0] = S_0
for i in range(1,step + 1):
  for j in range(1,i + 1):
    tree[j - 1, i] = tree[j - 1, i - 1]*u
    tree[j, i] = tree[j - 1, i - 1]*d
print(np.around(tree,3))

[[100.   113.31 128.4  145.5  164.87]
 [  0.    88.25 100.   113.31 128.4 ]
 [  0.     0.    77.88  88.25 100.  ]
 [  0.     0.     0.    68.73  77.88]
 [  0.     0.     0.     0.    60.65]]


In [128]:
#American call
#No dividends -> Implies same value as European Option -> Simply Discount Back End Payoffs
K = 100
exercise_payoff = tree - K
print(np.around(exercise_payoff,3))

[[   0.     13.31   28.4    45.5    64.87]
 [-100.    -11.75    0.     13.31   28.4 ]
 [-100.   -100.    -22.12  -11.75    0.  ]
 [-100.   -100.   -100.    -31.27  -22.12]
 [-100.   -100.   -100.   -100.    -39.35]]


In [129]:
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
print(np.around(exercise_payoff,3))

[[ 0.   13.31 28.4  45.5  64.87]
 [ 0.    0.    0.   13.31 28.4 ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]


In [130]:
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
print(np.around(cashflows,3))

[[ 0.    0.    0.    0.   64.87]
 [ 0.    0.    0.    0.   28.4 ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]


In [131]:
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = (q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T)
print(np.around(cashflows,3))

[[11.74 19.37 30.87 46.74 64.87]
 [ 0.    3.82  7.46 14.56 28.4 ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]


In [135]:
#European Put, K = $120
K = 120
exercise_payoff = K - tree
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = (q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T)
print(np.around(cashflows,3))

[[19.39 11.46  4.51  0.    0.  ]
 [ 0.   28.46 19.24  9.5   0.  ]
 [ 0.    0.   39.16 30.26 20.  ]
 [ 0.    0.    0.   49.78 42.12]
 [ 0.    0.    0.    0.   59.35]]


In [138]:
#American Put, K = $120
#Need to compare value with exercise payoff
K = 120
exercise_payoff = K - tree
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = max((q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T),exercise_payoff[j - 1, i - 1])
print(np.around(cashflows,3))

[[21.14 11.81  4.51  0.    0.  ]
 [ 0.   31.75 20.    9.5   0.  ]
 [ 0.    0.   42.12 31.75 20.  ]
 [ 0.    0.    0.   51.27 42.12]
 [ 0.    0.    0.    0.   59.35]]


In [140]:
#Incorporating dividends quarterly at 5%
div = 0.05
tree = np.zeros((step + 1,step + 1))
tree[0,0] = S_0
for i in range(1,step + 1):
  for j in range(1,i + 1):
    tree[j - 1, i] = tree[j - 1, i - 1]*u*(1-div)
    tree[j, i] = tree[j - 1, i - 1]*d*(1-div)
print(np.around(tree,3))

[[100.   107.65 115.88 124.75 134.29]
 [  0.    83.84  90.25  97.15 104.58]
 [  0.     0.    70.29  75.66  81.45]
 [  0.     0.     0.    58.93  63.43]
 [  0.     0.     0.     0.    49.4 ]]


In [147]:
#At-The-Money American Call
K = 100
exercise_payoff = tree - K
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = max((q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T),exercise_payoff[j - 1, i - 1])
print(np.around(cashflows,3))

[[ 4.76  8.71 15.88 24.75 34.29]
 [ 0.    0.62  1.2   2.35  4.58]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]]


In [145]:
#European Put, K = $120
K = 120
exercise_payoff = K - tree
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = (q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T)
print(np.around(cashflows,3))

[[33.68 25.21 16.21  7.32  0.  ]
 [ 0.   43.7  35.59 26.21 15.41]
 [ 0.    0.   53.6  46.63 38.55]
 [ 0.    0.    0.   62.53 56.57]
 [ 0.    0.    0.    0.   70.6 ]]


In [146]:
#American Put, K = $120
#Need to compare value with exercise payoff
K = 120
exercise_payoff = K - tree
for i in range(step + 1):
  for j in range(step + 1):
    exercise_payoff[i, j] = max(0, exercise_payoff[i, j])
cashflows = np.zeros((step + 1,step + 1))
cashflows[:,step] = exercise_payoff[:,step]
for i in range(step, -1, -1):
  for j in range(i, 0, -1):
    cashflows[j - 1, i - 1] = max((q*cashflows[j - 1, i] + (1-q)*cashflows[j,i])*math.exp(-r*delta_T),exercise_payoff[j - 1, i - 1])
print(np.around(cashflows,3))

[[33.68 25.21 16.21  7.32  0.  ]
 [ 0.   43.7  35.59 26.21 15.41]
 [ 0.    0.   53.6  46.63 38.55]
 [ 0.    0.    0.   62.53 56.57]
 [ 0.    0.    0.    0.   70.6 ]]
